<a href="https://colab.research.google.com/github/kittimaxz/Study/blob/main/Test_CronJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-analytics-data
!pip install mysql-connector

In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Metric, Dimension, RunReportRequest
import mysql.connector

In [ ]:
# สร้างไคลเอนต์เพื่อเชื่อมต่อกับ GA4
client = BetaAnalyticsDataClient()
client

In [ ]:
# กำหนดการร้องขอข้อมูล
request = RunReportRequest(
    property="properties/YOUR_PROPERTY_ID",  # ใส่ Property ID ของ GA4
    dimensions=[Dimension(name="city"), Dimension(name="country")],
    metrics=[Metric(name="activeUsers")],
    date_ranges=[DateRange(start_date="2023-01-01", end_date="today")]
)
request

In [ ]:
# รันการดึงข้อมูล
response = client.run_report(request)
response

In [ ]:
# พิมพ์ข้อมูลออกมา
for row in response.rows:
    print(row)

In [ ]:
# เชื่อมต่อกับ MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="yourusername",
    password="yourpassword",
    database="analytics_db"
)
connection

In [ ]:
cursor = connection.cursor()
cursor

In [ ]:
# สมมติว่ามีการสร้างตารางล่วงหน้าแล้ว
sql = "INSERT INTO analytics_data (city, country, active_users) VALUES (%s, %s, %s)"
sql

In [ ]:
for row in response.rows:
    data = (row.dimension_values[0].value, row.dimension_values[1].value, row.metric_values[0].value)
    cursor.execute(sql, data)

In [ ]:
connection.commit() # save การเปลี่ยนแปลง
connection.close() # ปิดการเชื่อมต่อ